<a href="https://colab.research.google.com/github/poliduckie/ReinforcementLearning/blob/main/PPO_duckie_baseline3_multiple_maps_birdeye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os 
branch = "daffy" #@param ['master', 'daffy']
if branch == "daffy":
  !pip install duckietown-gym-daffy
else:
  if not os.path.isdir('gym-duckietown') and not os.path.isdir('../gym-duckietown'):
    !git clone --branch {branch} https://github.com/duckietown/gym-duckietown.git
    !pip3 install -e gym-duckietown
  if "/gym-duckietown" not in os.getcwd():
    os.chdir('gym-duckietown')
!apt install xvfb -y
!pip3 install pyvirtualdisplay

from pyvirtualdisplay import Display
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
%matplotlib inline

## used to copy the model to the gdrive folder
import shutil

!apt update && apt install cmake libopenmpi-dev python3-dev zlib1g-dev
!pip install stable-baselines3[extra]

## imports for the ObsWrapper
import cv2
import gym
from gym.spaces import Box

import gym_duckietown

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.9).
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backport

DEBUG:commons:version: 6.2.4 *
DEBUG:typing:version: 6.2.3
DEBUG:duckietown_world:duckietown-world version 6.2.38 path /usr/local/lib/python3.7/dist-packages
DEBUG:geometry:PyGeometry-z6 version 2.1.4 path /usr/local/lib/python3.7/dist-packages
DEBUG:aido_schemas:aido-protocols version 6.0.59 path /usr/local/lib/python3.7/dist-packages
DEBUG:nodes:version 6.2.13 path /usr/local/lib/python3.7/dist-packages pyparsing 2.4.7
DEBUG:gym-duckietown:gym-duckietown version 6.1.30 path /usr/local/lib/python3.7/dist-packages



{'audio': ('directsound', 'openal', 'pulse', 'silent'), 'debug_font': False, 'debug_gl': True, 'debug_gl_trace': False, 'debug_gl_trace_args': False, 'debug_graphics_batch': False, 'debug_lib': False, 'debug_media': False, 'debug_texture': False, 'debug_trace': False, 'debug_trace_args': False, 'debug_trace_depth': 1, 'debug_trace_flush': True, 'debug_win32': False, 'debug_x11': False, 'graphics_vbo': True, 'shadow_window': True, 'vsync': None, 'xsync': True, 'xlib_fullscreen_override_redirect': False, 'darwin_cocoa': True, 'search_local_libs': True}


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [5]:
!apt update && apt install cmake libopenmpi-dev python3-dev zlib1g-dev
!pip install stable-baselines3[extra]
!pip install pyparsing==2.4.7

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [6]:
map_name = "Duckietown-zigzag_dists-v0" #@param ['Duckietown-straight_road-v0','Duckietown-4way-v0','Duckietown-udem1-v0','Duckietown-small_loop-v0','Duckietown-small_loop_cw-v0','Duckietown-zigzag_dists-v0','Duckietown-loop_obstacles-v0','Duckietown-loop_pedestrians-v0']

In [7]:
import cv2
import gym
from gym.spaces import Box

cutted_img_height = 350 #@param {type: "slider", min: 0, max: 480, step:1}
resize_ratio = 0.35 #@param {type: "slider", min: 0.0, max: 1.0, step:0.01}

img_height = 480
top_crop = img_height - cutted_img_height

img_final_height = int(cutted_img_height * resize_ratio)
img_final_width = int(640 * resize_ratio)

def cropimg(img):
    """
    Crop top of image top_crop px, they are noise most of the time

    :param img: (RGB image as np array) Image to be cropped
    """
    return img[top_crop:,:]

def houghtransform(img):
    """
    Apply Hough Line transform, for theory see:
    https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html

    :param img: (RGB image as np array)
    """
    frame_BGR = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    #gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY, 3)
    edges = cv2.Canny(frame_BGR,50,150,apertureSize = 3)
    #minLineLength = 100
    #maxLineGap = 10
    #lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
    #for x1,y1,x2,y2 in lines[0]:
    #    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
    imgRGB = cv2.cvtColor(edges, cv2.COLOR_BGR2RGB)
    return imgRGB

def resizeimg(img, ratio):
    """
    Resize image
    :param img: (np array)
    :param ratio: (float) 0<ratio<1
    """
    return cv2.resize(img, (0,0), fx=ratio, fy=ratio) 
  
def takeyellow(img):
    """
    Extract yellow lines, for color ranges see:
    https://stackoverflow.com/questions/48109650/how-to-detect-two-different-colors-using-cv2-inrange-in-python-opencv

    :param img: (RGB image as np array)
    """
    frame_HSV = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    frame_threshold = cv2.inRange(frame_HSV, (20,100,100), (50, 255, 255))
    imgRGB = cv2.cvtColor(frame_threshold, cv2.COLOR_GRAY2RGB)
    return imgRGB

def takewhiteyellow(img):
    """
    Extract white and yellow lines

    :param img: (RGB image as np array)
    """
    #white
    sensitivity = 100
    lower_white = np.array([0,0,255-sensitivity])
    upper_white = np.array([255,sensitivity,255])
    frame_HSV = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    maskwhite = cv2.inRange(frame_HSV, lower_white, upper_white)
    img[maskwhite > 0] = (255, 0, 0)
    img[maskwhite == 0] = (0,0,0)
    #yellow
    maskyellow = cv2.inRange(frame_HSV, (15,70,70), (50, 255, 255))
    img[maskyellow > 0] = (0, 255, 0)
    return img

def white_balance(img):
    """
    Grayworld assumption:
    https://stackoverflow.com/questions/46390779/automatic-white-balancing-with-grayworld-assumption/46391574

    :param img: (RGB image as np array)
    """
    result = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv2.cvtColor(result, cv2.COLOR_LAB2RGB)
    return result

def birdeye(img, test=False):
    """
    Apply perspective transform to image, for theory see:
    https://docs.opencv.org/3.4/da/d6e/tutorial_py_geometric_transformations.html

    Also:
    https://stackoverflow.com/questions/48264861/birds-eye-view-opencv
    """
    black_wd = int(2500*resize_ratio)
    black_img = np.zeros(shape=(img.shape[0], black_wd, 3), dtype=np.uint8)
    x_offset=int((black_wd-img.shape[0])/2)
    black_img[:img.shape[0], x_offset:x_offset+img.shape[1]] = img
    img = black_img
    row, cols, ch = img.shape
    src = np.float32([[1060, 170], [1600, 170], [110, row/resize_ratio], [cols/resize_ratio, row/resize_ratio]])*resize_ratio
    dst = np.float32([[0,0],[img_final_width,0],[0,img_final_height],[img_final_width,img_final_height]])
    M = cv2.getPerspectiveTransform(src, dst)
    img_transformed = cv2.warpPerspective(img, M, (cols, row))
    img_cutted_to_480_640 = img_transformed[:img_final_height, :img_final_width]
    return img_cutted_to_480_640


class ObsWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(ObsWrapper, self).__init__(env)
        self.observation_space = Box(0, 255, (img_final_height, img_final_width, 3), dtype=self.observation_space.dtype)
        self.accept_start_angle_deg = 4
        self.env = env

    def observation(self, obs):
        # cropped = cropimg(obs)
        balanced = white_balance(obs)
        img = takewhiteyellow(balanced)
        resized = resizeimg(img, resize_ratio)
        bird = birdeye(resized)
        return bird

In [8]:
class NormalizeWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(NormalizeWrapper, self).__init__(env)
        self.observation_space = Box(0, 1, self.observation_space.shape, dtype=self.observation_space.dtype)
        self.env = env

    def observation(self, obs):
        # cropped = cropimg(obs)
        balanced = white_balance(obs)
        img = takewhiteyellow(balanced)
        bird = birdeye(img)
        resized = resizeimg(bird, resize_ratio)
        return bird

In [9]:
import gym, gym_duckietown

display = Display(visible=0, size=(1400, 900))
display.start()

env = gym.make(map_name)
# env = BasicWrapper(env)
env = ObsWrapper(env)

INFO:gym-duckietown:Information about the graphics card:
 pyglet_version: 1.5.0
    information: dict[4]
                 │ vendor: VMware, Inc.
                 │ renderer: llvmpipe (LLVM 10.0.0, 256 bits)
                 │ version: 3.1 Mesa 20.0.8
                 │ shading-language-version: 1.40
  nvidia_around: True
DEBUG:gym-duckietown:Falling back to non-multisampled frame buffer
DEBUG:gym-duckietown:Falling back to non-multisampled frame buffer
INFO:duckietown_world: data: /usr/local/lib/python3.7/dist-packages/duckietown_world/data
DEBUG:gym-duckietown:loading map file "/usr/local/lib/python3.7/dist-packages/duckietown_world/data/gd1/maps/zigzag_dists.yaml"
INFO:gym-duckietown:done
DEBUG:gym-duckietown:loading mesh 'duckiebot' from file_path '/usr/local/lib/python3.7/dist-packages/duckietown_world/data/gd1/meshes/duckiebot/duckiebot.obj'
DEBUG:gym-duckietown:loading materials from /usr/local/lib/python3.7/dist-packages/duckietown_world/data/gd1/meshes/duckiebot/duckiebot.mtl
D

In [10]:
class DtRewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super(DtRewardWrapper, self).__init__(env)

    def reward(self, reward):
        if reward == -1000:
            reward = -10
        elif reward > 0:
            reward += 10
        else:
            reward += 4

        return reward

In [11]:
#env = DtRewardWrapper(env)

In [12]:
class ActionWrapper(gym.ActionWrapper):
    """
    Wrapper to change the range of possible actions.
    """
    def __init__(self, env):
        super(ActionWrapper, self).__init__(env)
        # This line stops the car from moving backwards
        self.action_space = Box(low=np.array([0.1, -0.5]), high=np.array([1.0, 0.5]), dtype=np.float32)
        # Default:
        # self.action_space = Box(low=np.array([-1.0, -1.0]), high=np.array([1.0, 1.0]), dtype=np.float32)
        self.env = env
    
    def action(self, act):
        return act

In [13]:
class ActionNoiseWrapper(gym.ActionWrapper):
    """
    Wrapper to change the range of possible actions.
    """
    def __init__(self, env):
        super(ActionNoiseWrapper, self).__init__(env)
        self.env = env
    
    def action(self, act):
        #create noise with normal distribution(mean 0, 0.01 std)
        noise = np.random.normal(0, 0.01, act.shape)
        #add noise to the signal
        act = act + noise
        return act

In [14]:
env = ActionWrapper(env)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [15]:
%tensorboard --logdir ./ppo_duckieloop/

Reusing TensorBoard on port 6006 (pid 2312), started 4:31:32 ago. (Use '!kill 2312' to kill it.)

<IPython.core.display.Javascript object>

In [16]:
import os
from stable_baselines3.common.monitor import Monitor

# Create log dir
log_dir = "./ppo_duckieloop/"
os.makedirs(log_dir, exist_ok=True)

# Logs will be saved in log_dir/monitor.csv
env = Monitor(env, log_dir)

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

from stable_baselines3.common.evaluation import evaluate_policy

# MultiInputPolicy does not work with this env:
# https://github.com/hill-a/stable-baselines/issues/417

model = PPO(
    "MlpPolicy",
    env,
    verbose=False,
    tensorboard_log="./ppo_duckieloop/",
    gamma=0.99,
    # https://arxiv.org/abs/2005.05719
    use_sde=True,
    # https://www.researchgate.net/figure/TD3-Hyperparameters_tbl2_341341608
    sde_sample_freq=64, # def: -1
    # https://github.com/hill-a/stable-baselines/issues/213
    # https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence
    # target kl sets an early stopping limit if the kl diverges too much
    # target_kl
    )


In [ ]:
load = True
if load:
  model = model = PPO.load("ppoDuckietown-zigzag_dists-v0140000.0")

In [ ]:
run = "True" #@param ['True', 'False']

In [ ]:
map_name = "Duckietown-zigzag_dists-v0" #@param ['Duckietown-straight_road-v0','Duckietown-4way-v0','Duckietown-udem1-v0','Duckietown-small_loop-v0','Duckietown-small_loop_cw-v0','Duckietown-zigzag_dists-v0','Duckietown-loop_obstacles-v0','Duckietown-loop_pedestrians-v0']


In [24]:
N_ITERATIONS = 20
## maps without obstacles
map_names = ['Duckietown-straight_road-v0','Duckietown-straight_road-v0','Duckietown-straight_road-v0','Duckietown-small_loop-v0','Duckietown-small_loop_cw-v0','Duckietown-small_loop-v0','Duckietown-small_loop_cw-v0','Duckietown-small_loop-v0','Duckietown-small_loop_cw-v0','Duckietown-zigzag_dists-v0','Duckietown-zigzag_dists-v0','Duckietown-zigzag_dists-v0','Duckietown-zigzag_dists-v0','Duckietown-zigzag_dists-v0','Duckietown-zigzag_dists-v0','Duckietown-zigzag_dists-v0','Duckietown-zigzag_dists-v0','Duckietown-zigzag_dists-v0','Duckietown-zigzag_dists-v0','Duckietown-zigzag_dists-v0']
if run=="True":
  print("Starting...")
  for time in range(N_ITERATIONS):
    env_new = gym.make(map_names[time%len(map_names)])
    env_new = ObsWrapper(env_new)
    env_new = ActionWrapper(env_new)
    env_new = Monitor(env_new, log_dir)
    model.set_env(env_new)
    if run=="True":
      model.learn(total_timesteps=int(1e4), tb_log_name="run_"+str(time))
      mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
      model.save("ppo"+map_name+str(1e4*time))
      print(f"#{time} Trained 10000 timesteps, mean_reward: {mean_reward}, std_reward: {std_reward}")
    else:
      break  
  ipythondisplay.clear_output(wait=True)

INFO:gym-duckietown:Information about the graphics card:
 pyglet_version: 1.5.0
    information: dict[4]
                 │ vendor: VMware, Inc.
                 │ renderer: llvmpipe (LLVM 10.0.0, 256 bits)
                 │ version: 3.1 Mesa 20.0.8
                 │ shading-language-version: 1.40
  nvidia_around: True
DEBUG:gym-duckietown:Falling back to non-multisampled frame buffer
DEBUG:gym-duckietown:Falling back to non-multisampled frame buffer
DEBUG:gym-duckietown:loading map file "/usr/local/lib/python3.7/dist-packages/duckietown_world/data/gd1/maps/straight_road.yaml"
INFO:gym-duckietown:done


Starting...


DEBUG:gym-duckietown:No tile found at [-0.02871147  0.          0.30263937] (-1, 0)
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [0.05438121 0.         0.35364776]
DEBUG:gym-duckietown:l_pos: [0.1374739  0.         0.40465615]
DEBUG:gym-duckietown:r_pos: [-0.02871147  0.          0.30263937]
DEBUG:gym-duckietown:f_pos: [-0.00682885  0.          0.45335898]
DEBUG:gym-duckietown:No tile found at [-0.03760193  0.          0.20864481] (-1, 0)
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [0.05309013 0.         0.24443867]
DEBUG:gym-duckietown:l_pos: [-0.03760193  0.          0.20864481]
DEBUG:gym-duckietown:r_pos: [0.14378219 0.         0.28023253]
DEBUG:gym-duckietown:f_pos: [0.09604276 0.         0.1356082 ]
DEBUG:gym-duckietown:No tile found at [0.3676835  0.         0.59911307] (0, 1)
DEB

KeyboardInterrupt: ignored

In [ ]:
env_new = gym.make(map_name)
env_new = ObsWrapper(env_new)
env_new = Monitor(env_new, log_dir)

INFO:gym-duckietown:Information about the graphics card:
 pyglet_version: 1.5.0
    information: dict[4]
                 │ vendor: VMware, Inc.
                 │ renderer: llvmpipe (LLVM 10.0.0, 256 bits)
                 │ version: 3.1 Mesa 20.0.8
                 │ shading-language-version: 1.40
  nvidia_around: True


In [1]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: DtRewardWrapper(ObsWrapper(gym.make(map_name, accept_start_angle_deg=4)))])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, reward, done, _ = eval_env.step(action)
    if done and reward < 0:
      print("*** CRASHED ***")
      print(reward)
    elif done:
      print("SAFE")
      print(reward)
    

  # Close the video recorder
  eval_env.close()

In [ ]:
load = False

if load:
    time_to_load = 8
    model_to_be_loaded = "ppo"+map_name+str(1e4*time_to_load)
    model = PPO.load(model_to_be_loaded)
record_video(model, video_length=500, prefix=map_name)

In [ ]:
if True:
  show_videos('videos', prefix=map_name)

In [ ]:
done = False
obs = env.reset()
actions = []
while not done:
  action, _ = model.predict(obs)
  obs, rewad, done, _ = env.step(action)
  actions.append(action)

In [ ]:
actions[:10]

In [ ]:
env.action_space

In [ ]:
env.action_space.sample()